## Env setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# https://spacy.io/usage/
!pip install spacy_transformers
!pip install -U spacy

!pip install pdfminer-six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 35.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

In [7]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [8]:
!nvidia-smi

Mon Jun 10 06:12:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
spacy.__version__

'3.7.5'

## loading data

In [29]:
data = json.load(open('/content/drive/MyDrive/resume-parser/data/dataset.json', 'r'))

JSONDecodeError: Invalid control character at: line 6262 column 16 (char 207415)

In [ ]:
len(data)

In [ ]:
data[0]

In [27]:
!python -m spacy init fill-config /content/drive/MyDrive/resume-parser/data/base_config.cfg /content/drive/MyDrive/resume-parser/data/config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/resume-parser/data/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


## training model

In [16]:
def get_spacy_doc(data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot: # check for overlap of indeies
        skip_entity = False
        for i in range(start, end):
            if i in entity_indices:
                skip_entity = True
                break

        if skip_entity:
            continue

        entity_indices = entity_indices + list(range(start, end))

        try:
            span = doc.char_span(start, end, label=label, alignment_mode="strict")

        except:
            continue

        if span is None:
            continue
        else:
            ents.append(span)

    try:
        doc.ents = ents
        db.add(doc)

    except:
        pass


  return db


In [17]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)

NameError: name 'data' is not defined

In [ ]:
len(train), len(test)

(811, 203)

In [ ]:
train_db = get_spacy_doc(train)
train_db.to_disk('/content/drive/MyDrive/resume-parser/data/train.spacy')

test_db = get_spacy_doc(test)
test_db.to_disk('/content/drive/MyDrive/resume-parser/data/test.spacy')


NameError: name 'get_spacy_doc' is not defined

In [ ]:
train_db.tokens, test_db.tokens

203

In [3]:
!python -m spacy \
    train /content/drive/MyDrive/resume-parser/data/config.cfg \
    --output ./content/drive/MyDrive/resume-parser/output \
    --paths.train /content/drive/MyDrive/resume-parser/data/train.spacy \
    --paths.dev /content/drive/MyDrive/resume-parser/data/test.spacy \
    --gpu-id 0

Usage: python -m spacy train [OPTIONS] CONFIG_PATH
Try 'python -m spacy train --help' for help.
╭─ Error ──────────────────────────────────────────────────────────────────────────────────────────╮
│ Invalid value for 'CONFIG_PATH': Path '/content/drive/MyDrive/resume-parser/data/config.cfg'     │
│ does not exist.                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯


## test


In [ ]:
model_path = '/content/drive/MyDrive/resume-parser/output/model-best'

In [ ]:
import spacy
nlp = spacy.load(model_path)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/spacy_transformers/layers/hf_shim.

In [ ]:
resume_path = "/content/drive/MyDrive/resume-parser/test/resume/resume1.pdf"
jd_path = '/content/drive/MyDrive/resume-parser/test/job-description/jd1.txt'

In [ ]:
import re
from pdfminer.high_level import extract_text

text = extract_text(resume_path)
text

'Karnam Shyam\n\nStudent, Aspiring Full-Stack Developer Powered by AI/ML Insights\n9346872174 | karnam.shyam2004@gmail.com | @linkedin | @github | @leetcode | @hackerrank\n\nSummary\n\nExperienced and results-driven professional with a strong background in full-stack Java development and a\npassion for leveraging Python for AI, ML, and deep learning applications. Proven track record demonstrated\nthrough research papers showcasing expertise in these areas. I am constantly seeking opportunities to expand my\nknowledge and skills through internships, workshops, and personal projects. I possess strong logical thinking and\nproblem-solving abilities. I believe in continuous learning and strive to stay updated with the latest industry\ntrends.\n\nTechnical Skills\n\nLanguages: Python, Java, C, HTML, CSS, JavaScript, SQL(MySQL, SQLite, Oracle), MongoDB\nFrameworks: Flask, SpringBoot, Angular16, Bootstrap, Hibernate\nDeveloper Tools: Git, Firebase, Maven\nLibraries: Tensorflow, sklearn, numpy

In [ ]:
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, " ->>> " ,ent.label_)

NameError: name 'nlp' is not defined

In [ ]:
resp = {}
for ent in doc.ents:
    if resp.get(ent.label_) is None:
        resp[ent.label_] = [ent.text]
    else:
        resp[ent.label_].append(ent.text)

resp

{'NAME': ['Karnam Shyam'],
 'SKILLS': ['Python',
  'Java',
  'C',
  'HTML',
  'CSS',
  'JavaScript',
  'MongoDB',
  'Flask',
  'SpringBoot',
  'Angular16',
  'Bootstrap',
  'Hibernate',
  'Git',
  'Firebase',
  'Maven',
  'Tensorflow',
  'sklearn',
  'numpy',
  'pandas',
  'Arduino',
  'Flask',
  'Integrated MTech CSE'],
 'UNIVERSITY': ['VIT-AP University']}

In [ ]:
import re

def extract_contact_number_from_resume(text):
    contact_number = None

    # Use regex pattern to find a potential contact number
    pattern = r"\b(?:\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b"
    match = re.search(pattern, text)
    if match:
        contact_number = match.group()

    return contact_number

phone = extract_contact_number_from_resume(text)
phone

'9346872174'

In [ ]:
import re

def extract_email_from_resume(text):
    email = None

    # Use regex pattern to find a potential email address
    pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"
    match = re.search(pattern, text)
    if match:
        email = match.group()

    return email

email = extract_email_from_resume(text)
email

'karnam.shyam2004@gmail.com'

In [ ]:
jd = open(jd_path, 'r').read()
jd

"Python LLM Data Scientist\n\nA US-based AI client looking for a Python LLM Data Scientist/Analyst.\n\nThey're looking for someone to take on the responsibility of guiding peer code reviews, nurturing an atmosphere where constructive feedback is exchanged to refine our code and foster professional growth.\n\nYour Role:\n\nBuild Python codes for big challenges that can grow with us.\nDig into free data out there and find cool insights.\nWork side-by-side with our researchers to hit our goals.\nUse data to crack tough work-related questions.\n\nYou Should Have:\n\nA degree in stuff like Engineering or Computer Science, or you're just really good at this.\nMust have worked in data science or analysis for 1 year.\nBeen coding with Python for as a professional.\nSolid skills in data science and analysis.\nReally good English, talking and writing.\n\nNice to have:\n\nIf you know SQL, that's a plus.\nSuper good at talking and teaming up with people.\nThe ability to think through data and make

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def tfidf_matcher(job_description, resumes):
    vectorizer = TfidfVectorizer().fit_transform([job_description, resumes])
    vectors = vectorizer.toarray()

    job_vector = vectors[0]
    resume_vectors = vectors[1:]
    similarities = cosine_similarity([job_vector], resume_vectors)[0]
    print(similarities)

    return similarities

In [ ]:
def count_matcher(job_description, resumes):
    vectorizer = CountVectorizer().fit_transform([job_description, resumes])
    vectors = vectorizer.toarray()

    job_vector = vectors[0]
    resume_vectors = vectors[1:]
    similarities = cosine_similarity([job_vector], resume_vectors)[0]
    print(similarities)

    return similarities

In [ ]:
tfidf_matcher(jd, text)

[0.21611968]


array([0.21611968])

In [ ]:
count_matcher(jd, text)

[0.33672777]


array([0.33672777])